## Descripción

Un avión vuela con velocidad $V$ a $H_a$ metros del suelo en dirección a un cañón como se muestra en la figura.

El cañón tiene forma de trapecio isósceles invertido, de una altura $H_c$, largo $L$ y ángulo $\alpha$ con respecto al horizonte.

Cuando se encuentra a una distancia $d$ del borde del cañón, el avión suelta una bomba.

![Cañón y trayectoria del avión](im2.png)


In [3]:
import matplotlib
matplotlib.use('TkAgg')  # Cambia el backend si es necesario
import matplotlib.pyplot as plt
import numpy as np
from matplotlib.offsetbox import OffsetImage, AnnotationBbox
import tkinter as tk
import signal
import sys

# Constante de gravedad
g = 9.81

# Función de la trayectoria de la bomba
def trayectoria(x, x0, y0, v0x, v0y):
    t = (x - x0) / v0x
    return y0 - 0.5 * g * t**2

def simular():
    # Obtener los valores de los parámetros desde la interfaz gráfica
    ha = float(entry_ha.get())
    hc = float(entry_hc.get())
    L = float(entry_L.get())
    alpha = np.radians(float(entry_alpha.get()))
    d = float(entry_d.get())
    v0 = float(entry_v0.get())
    pos_crater = float(entry_pos_crater.get())

    # Cerrar la ventana de entrada
    root.destroy()

    # Coordenadas del cráter (entre y=100 y y=0)
    x_crater = [
        pos_crater,
        pos_crater + (100 / np.tan(alpha)),
        pos_crater + L - (100 / np.tan(alpha)),
        pos_crater + L
    ]
    y_crater = [100, 0, 0, 100]

    # Coordenadas del suelo
    x_suelo_izq = [0, pos_crater]
    y_suelo_izq = [100, 100]
    x_suelo_der = [pos_crater + L, 4000]
    y_suelo_der = [100, 100]

    # Crear figura y ejes
    fig, ax = plt.subplots(figsize=(12, 6))
    ax.set_xlim(0, 4000)
    ax.set_ylim(-50, ha + 50)

    # Dibujar el cráter
    ax.fill(x_crater, y_crater, color="lightgrey", label="Cráter (trapecio invertido)", alpha=0.5)
    
    # Dibujar las paredes diagonales del cráter con líneas continuas
    ax.plot([x_crater[0], x_crater[1]], [y_crater[0], y_crater[1]], color='black', linestyle='-', linewidth=2, label="Pared izquierda del cráter")
    ax.plot([x_crater[2], x_crater[3]], [y_crater[2], y_crater[3]], color='black', linestyle='-', linewidth=2, label="Pared derecha del cráter")


    # Dibujar el suelo
    ax.plot(x_suelo_izq, y_suelo_izq, color="brown", linewidth=2, label="Suelo izquierdo")
    ax.plot(x_suelo_der, y_suelo_der, color="brown", linewidth=2, label="Suelo derecho")

    # Ruta de la imagen del avión (asegúrate de que la imagen esté en el mismo directorio que este script)
    avion_imagen_ruta = r"avion.png"
    img = plt.imread(avion_imagen_ruta)

    # Añadir el marcador del avión en la posición inicial
    avion_marker = AnnotationBbox(OffsetImage(img, zoom=0.1), (0, ha), frameon=False)
    ax.add_artist(avion_marker)

    # Elementos de la bomba y trayectoria
    bomba, = ax.plot([], [], 'ro', label="Bomba", markersize=8)
    trayectoria_line, = ax.plot([], [], color="orange", label="Trayectoria de la bomba")

    # Posiciones del avión
    posiciones = np.linspace(0, 4000, 200)  # Movimiento suave del avión

    # Variables de la bomba
    bomba_lanzada = False
    x_bomba = []
    y_bomba = []

    # Simulación del movimiento del avión y la caída de la bomba
    for x_avion in posiciones:
        # Mover el avión
        avion_marker.xybox = (x_avion, ha)

        # Lanzar la bomba cuando el avión alcanza la distancia d
        if x_avion >= d and not bomba_lanzada:
            bomba_lanzada = True
            x_bomba = [d]
            y_bomba = [ha]

        # Actualizar la trayectoria de la bomba si fue lanzada
        if bomba_lanzada:
            # Calcular el siguiente punto de la bomba
            x_actual = x_bomba[-1] + v0 * 0.05  # Incremento en x basado en velocidad
            y_actual = trayectoria(x_actual, d, ha, v0, 0)
            
            
             # Colisión con la pared izquierda del cráter
            if (x_crater[0] <= x_actual <= x_crater[1]) and y_actual <= 100 - (x_actual - x_crater[0]) * np.tan(alpha):
                y_actual = 100 - (x_actual - x_crater[0]) * np.tan(alpha)
                x_bomba.append(x_actual)
                y_bomba.append(y_actual)
                bomba.set_data([x_actual], [y_actual])
                trayectoria_line.set_data(x_bomba, y_bomba)
                break  # Detener la simulación

            # Colisión con la pared derecha del cráter
            elif (x_crater[2] <= x_actual <= x_crater[3]) and y_actual <= (x_crater[3] - x_actual) * np.tan(alpha):
                y_actual = (x_crater[3] - x_actual) * np.tan(alpha)
                x_bomba.append(x_actual)
                y_bomba.append(y_actual)
                bomba.set_data([x_actual], [y_actual])
                trayectoria_line.set_data(x_bomba, y_bomba)
                break  # Detener la simulación

            # Impacto en el fondo del cráter
            elif (x_crater[1] <= x_actual <= x_crater[2]) and y_actual <= 0:
                y_actual = 0
                x_bomba.append(x_actual)
                y_bomba.append(y_actual)
                bomba.set_data([x_actual], [y_actual])
                trayectoria_line.set_data(x_bomba, y_bomba)
                break  # Detener la simulación
            
            
            # Impacto con el suelo
            elif y_actual <= 100 and not (x_crater[0] <= x_actual <= x_crater[3]):
                y_actual = 100
                x_bomba.append(x_actual)
                y_bomba.append(y_actual)
                bomba.set_data([x_actual], [y_actual])
                trayectoria_line.set_data(x_bomba, y_bomba)
                break  # Detener la simulación

            # Agregar el punto a la trayectoria
            x_bomba.append(x_actual)
            y_bomba.append(y_actual)

            # Actualizar los datos de la bomba
            trayectoria_line.set_data(x_bomba, y_bomba)
            bomba.set_data([x_bomba[-1]], [y_bomba[-1]])

        # Dibujar el avión y actualizar la animación
        plt.legend()
        plt.title(f"Movimiento del avión y trayectoria de la bomba (Posición: {x_avion:.2f} m)")
        plt.xlabel("Distancia horizontal (m)")
        plt.ylabel("Altura (m)")
        plt.draw()
        plt.pause(0.01)  # Pausa para que el movimiento sea visible

    plt.show()

def signal_handler(sig, frame):
    print("Interrupción detectada. Cerrando el programa...")
    plt.close('all')
    try:
        root.quit()
        root.destroy()
    except tk.TclError:
        pass
    sys.exit(0)

# Configurar el manejador de señales
signal.signal(signal.SIGINT, signal_handler)

# Crear la ventana principal de Tkinter
root = tk.Tk()
root.title("Simulación de Caída de Bomba")
root.geometry("800x400")  # Tamaño de la ventana
root.configure(bg='green')  # Color verde



# Crear etiquetas y campos de entrada para los parámetros con valores por defecto
tk.Label(root, text="Altura inicial del avión (ha)", bg='green').grid(row=0)
tk.Label(root, text="Altura del fondo del cráter (hc)", bg='green').grid(row=1)
tk.Label(root, text="Longitud horizontal del cráter (L)", bg='green').grid(row=2)
tk.Label(root, text="Ángulo de las paredes del cráter (alpha)", bg='green').grid(row=3)
tk.Label(root, text="Distancia inicial del avión desde el borde del cráter (d)", bg='green').grid(row=4)
tk.Label(root, text="Velocidad inicial de la bomba (v0)", bg='green').grid(row=5)
tk.Label(root, text="Posición del cráter (pos_crater)", bg='green').grid(row=6)

entry_ha = tk.Entry(root)
entry_hc = tk.Entry(root)
entry_L = tk.Entry(root)
entry_alpha = tk.Entry(root)
entry_d = tk.Entry(root)
entry_v0 = tk.Entry(root)
entry_pos_crater = tk.Entry(root)

entry_ha.insert(10, "300")  
entry_hc.insert(10, "100")
entry_L.insert(10, "1500")
entry_alpha.insert(10, "45")
entry_d.insert(10, "100")
entry_v0.insert(10, "300")
entry_pos_crater.insert(10, "1500")

entry_ha.grid(row=0, column=1)
entry_hc.grid(row=1, column=1)
entry_L.grid(row=2, column=1)
entry_alpha.grid(row=3, column=1)
entry_d.grid(row=4, column=1)
entry_v0.grid(row=5, column=1)
entry_pos_crater.grid(row=6, column=1)

# Botón para iniciar la simulación
tk.Button(root, text='Iniciar Simulación', command=simular).grid(row=7, column=1, pady=4)

# Iniciar el bucle principal de Tkinter
root.mainloop()


Interrupción detectada. Cerrando el programa...


SystemExit: 0